# CONSTRUCTING SPEED CURVES FROM MOVES

This is annotated file for code that constructs speed curves. This ONLY works on Python 2.7 due to the movespy.moves module. It should output one speed curve for each roadtype+policy combination (e.g. 4 roadtypes, 4 policies, = 16 speed curves)

In [ ]:
import movespy.moves
import os
import csv
import sys

# Vehicle distribution data

We define the distribution of vehicles in these 4 policy cases. The vehicle codes (31, 21, etc) are in the image below. The numbers are taken from the files found here https://paper.dropbox.com/doc/WAZE-API-5yK5F5OGXKAna1tGlJbYD - scroll down to the 'vehicle distribution data' section. We further define the age distribution for type 21 (passenger vehicles) and type 31 (light trucks) because that data is available to us. This is also in the dropbox link above in that same section, in a file posted by Pedro about a survey for age ranges.

In [1]:
ldv_dist = {31: 0.82671, 52: 0.009786, 53: 0.001312, 11: 0.145326, 42: 0.016866}
bus_dist = {21: 0.5239439, 31: 0.400313, 52: 0.004738, 53: 0.0006351, 11: 0.07037} 
taxi_dist = {21: 0.501567, 31: 0.412061, 52: 0.0048774, 53: 0.00065379, 11: 0.07243501, 42: 0.00840580}
baseline_dist = {21: 0.5197, 31: 0.39707, 52: 0.0047, 53: 0.00063, 11: 0.0698, 42: 0.0081}

type_21_ages = {0:0.003295129, 1:0.031375358, 2:0.053724928, 3:0.060315186, 4:0.065902579, 5:0.070916905, 6:0.070630372, 7:0.08008596, 8:0.076217765, 9:0.065902579, 10:0.068051576, 11:0.053581662, 12:0.047277937, 13:0.036532951, 14:0.039398281, 15:0.027936963, 16:0.026790831, 17:0.024498567, 18:0.025071633, 19:0.016332378, 20:0.013180516, 21:0.012034384, 22:0.009025788, 23:0.006303725, 24:0.002578797, 25:0.003581662, 26:0.002292264, 27:0.001719198, 28:0.002005731, 29:0.001002865, 30:0.000859599, 31:0, 32:0.000286533, 33:0.000573066, 34:0.000143266, 35:0.000286533, 36:0.000143266, 37:0}
type_31_ages = {0: 0.000886525, 1: 0.028368794, 2: 0.046985816, 3: 0.055407801, 4: 0.060283688, 5: 0.052526596, 6: 0.053191489, 7: 0.057624113, 8: 0.043661348, 9: 0.043218085, 10: 0.055407801, 11: 0.066932624, 12: 0.06981383, 13: 0.056515957, 14: 0.065824468, 15: 0.055407801, 16: 0.045212766, 17: 0.033244681, 18: 0.025044326, 19: 0.019281915, 20: 0.017952128, 21: 0.01285461, 22: 0.007092199, 23:0.008200355, 24: 0.005984043, 25: 0.004654255, 26: 0.00177305, 27: 0.001551418, 28: 0.002216312, 29: 0.001329787, 30: 0.000443262, 31: 0.000443262, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.000221631, 37: 0.000221631}

## Moves Input

Now, all we're doing is plugging the distributions above into movespy, and doing so for every combination of policy and roadtype. The assumptions u

In [6]:
dists = {'ldv': ldv_dist, 'bus':bus_dist, 'taxi':taxi_dist, 'base':baseline_dist}
roadtypes = [2, 3, 4, 5]
for policy, dist in dists.items():
    for road in roadtypes:
        writeFile = open(policy + str(road) + 'speed_curve.csv', 'w') #writes to something like 'ldv5speed_curve.csv'
        writer = csv.DictWriter(writeFile, fieldnames=['speed','CO2-Atm','CO2-Eq','CO','PM10','VOC','SO2','PM2.5','NOx','NH3'])
        for i in range(0, 81): #where i is the kph. This tests 0-80kph.
            
            #below is just the basic movespy setup according to https://github.com/RSGInc/movespy 
            #conversions are done to turn everything into miles, since movespy operates using mph/miles.
            #some manual testing will reveal that volume and length scale the data linearly. Double length = double emissions, same with volume. Tested using factors of 10 (0.01, 0.1, 10, 100)
            
            link = {}
            link_data = {}
            link_data['grade'] = 0
            link_data['length'] = 1 #units are in miles
            link_data['road_type'] = road
            link_data['source_distr'] = dist
            link_data['speed'] = i*0.62137 #conversion to mph
            link_data['volume'] = 100 
            link[1] = link_data
            
            #the day_type is 5, meaning 'average weekday'
            #the county is 9002. This is selected from manual inspection of the mysql tables (run 'select * from county', counties with 
            #IDs from 9001-9015 all fall within Mexico City boundaries. Manual testing showed that all of them return the same results, so we can arbitrarily select one)
            activity = {'age_distr': {},'day_type': 5,'county': 9002,'hour':4,'month': 9,'year': 2017,'links': link}
            activity['age_distr'][21] = type_21_ages
            activity['age_distr'][31] = type_31_ages

            options = {'detail':'average'}

            moves = movespy.moves.Moves(activity, options)

            emissions_out=moves.run()
            
            temp = [row for row in emissions_out]
            #following codes (2, 3, 87, etc) identified from manual inspection of mysql tables 
            #run 'select * from pollutant', look at pollutantIDs
            emission_list = [row for row in temp if (row['pollutant'] in [2, 3, 87, 31, 30, 55,56,100, 110])]
            pollutantMap = {55: 'CO2-Atm', 56: 'CO2-Eq', 2: 'CO', 3: 'NOx', 87: 'VOC', 31:'SO2', 30:'NH3', 100:'PM10', 110:'PM2.5'}
            write_dict = {'speed': i}
            for emission_dict in emission_list:
                write_dict[pollutantMap[emission_dict['pollutant']]] = emission_dict['quantity']  
            writer.writerow(write_dict)
        writeFile.close()

NameError: name 'movespy' is not defined